In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data=pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [ ]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

In [ ]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,accuracy_score ,f1_score,confusion_matrix
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.impute import SimpleImputer

In [ ]:
df_new1 = train_data.query("Pclass==1")
df_new2 = train_data.query("Pclass==2")
df_new3 = train_data.query("Pclass==3")
median1=df_new1['Age'].median()
median2=df_new2['Age'].median()
median3=df_new3['Age'].median()

In [ ]:
features = ["Pclass", "Sex", "SibSp", "Parch"]
train_data['Age']=np.float32(train_data['Age'])
test_data['Age']=np.float32(test_data['Age'])

label_encoder = LabelEncoder()
train_data['Sex'] = label_encoder.fit_transform(train_data['Sex'])
test_data['Sex'] = label_encoder.fit_transform(test_data['Sex'])

# Imputation
my_imputer = SimpleImputer()
train_data[features] = pd.DataFrame(my_imputer.fit_transform(train_data[features]))

# Imputation removed column names; put them back
train_data[features].columns = train_data[features].columns

def transform_columns(column):
    Age = column[0]
    Pclass = column[1]
    
    if(pd.isna(Age)):
       if(Pclass == 1):
            return median1
       elif(Pclass == 2):
            return median2   
       else:
            return median3  
    else:
       return Age

for age1 in train_data["Age"]:
    if(age1<=0):
        train_data['Age'] = train_data['Age'].replace([age1],NA)
        
for age1 in test_data["Age"]:
    if(age1<=0):
        test_data['Age'] = test_data['Age'].replace([age1],NA)
        
train_data['Age'] = train_data[['Age', 'Pclass']].apply(transform_columns, axis = 1)
test_data['Age'] = test_data[['Age', 'Pclass']].apply(transform_columns, axis = 1)
features = ["Pclass", "Sex", "SibSp", "Parch","Age"]

In [ ]:
X = np.float32(train_data[features])
y = np.float32(train_data["Survived"])
train_X,val_X,train_y,val_y=train_test_split(X,y,test_size=0.2,random_state=100)

**Random Forest Model**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(random_state=100)
model_rf.fit(train_X, train_y)
predictions = model_rf.predict(val_X)
mae_rf=mean_absolute_error(val_y,predictions)
print("confusion matrix:",confusion_matrix(val_y,predictions))
print("mae: ",mae_rf)
print("accuracy score: ",accuracy_score(val_y,predictions))
print("f1 score: ",f1_score(val_y,predictions))

**Decision tree**

In [ ]:

from sklearn.tree import DecisionTreeClassifier


model_dt = DecisionTreeClassifier(random_state=100)
model_dt.fit(train_X, train_y)
predictions = model_dt.predict(val_X)
mae_dt=mean_absolute_error(val_y,predictions)
print("confusion matrix:",confusion_matrix(val_y,predictions))
print("mae: ",mae_dt)
print("accuracy score: ",accuracy_score(val_y,predictions))
print("f1 score: ",f1_score(val_y,predictions))

***KNN***

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# feature scaling
sc_X=StandardScaler()
train_X=sc_X.fit_transform(train_X)
val_X=sc_X.fit_transform(val_X)

"""
# to get the value of n_neighbors
import math
print(math.sqrt(len(val_y)))

"""
model_knn = KNeighborsClassifier(n_neighbors=13,p=2,metric='euclidean')
model_knn.fit(train_X, train_y)
predictions = model_knn.predict(val_X)
mae_knn=mean_absolute_error(val_y,predictions)
print("confusion matrix:",confusion_matrix(val_y,predictions))
print("mae: ",mae_knn)
print("accuracy score: ",accuracy_score(val_y,predictions))
print("f1 score: ",f1_score(val_y,predictions))

***Submission***

The model with most accuracy is submitted..So the KNN model with the accuracy 82.68% and also f1 score 0.78014 is chosen.

In [ ]:
predictions_submission=model_knn.predict(test_data[features])
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions_submission})
print(output)
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")